In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

2022-04-14 21:07:05.383048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-14 21:07:05.383068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [1]:
def create_df(journey):
    comunio = pd.read_csv(f'../data/pruebas/only_comunio_stats_J{journey}.csv')
    clas = pd.read_excel(f'../data/classification_J_{journey}.xlsx', sheet_name=f'classification_J_{journey}', index_col='Unnamed: 0')
    cal = pd.read_csv('../data/pruebas/Season_21-22.csv')
    
    teams_dict = {'Athletic Club': 'Athletic Club' ,
                 'CA Osasuna':'Osasuna',
                 'Club Atlético de Madrid':'Atlético Madrid',
                 'Cádiz CF':'Cádiz',
                 'Deportivo Alavés':'Alavés',
                 'Elche CF':'Elche',
                 'FC Barcelona':'Barcelona',
                 'Getafe CF':'Getafe',
                 'Granada CF':'Granada',
                 'Levante UD':'Levante',
                 'RC Celta de Vigo':'Celta Vigo',
                 'RCD Espanyol de Barcelona':'Espanyol',
                 'RCD Mallorca':'Mallorca',
                 'Rayo Vallecano de Madrid':'Rayo Vallecano',
                 'Real Betis Balompié':'Betis',
                 'Real Madrid CF':'Real Madrid',
                 'Real Sociedad de Fútbol':'Real Sociedad',
                 'Sevilla FC':'Sevilla',
                 'Valencia CF':'Valencia',
                 'Villarreal CF':'Villarreal'}
    
    new_team_name = []
    for team in comunio.Team:
  
        for k,v in teams_dict.items():
       
            if k == team:
           
                new_team_name.append(v)
    
    comunio['Team'] = new_team_name
    
    points_per_team = comunio.groupby('Team').sum().reset_index()
    points_per_team= points_per_team.rename(columns={'Total_Points': 'Squad_Points',
                                                       'Points_Average': 'Squad_Average_Points',
                                                       'Avg_last_5_games': 'Squad_Avg_last_5_Games',
                                                       'Value': 'Value_Squad',
                                                       f'J_{journey -4}': f'Squad_Points_J_{journey -4}',
                                                       f'J_{journey -3}': f'Squad_Points_J_{journey -3}',
                                                       f'J_{journey -2}': f'Squad_Points_J_{journey -2}',
                                                       f'J_{journey -1}': f'Squad_Points_J_{journey -1}',
                                                       f'J_{journey}': f'Squad_Points_J_{journey}',})
    
    points_per_team = points_per_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_1 = comunio.merge(points_per_team, how='left', left_on='Team', right_on='Team')
    
    matches_J = cal.loc[cal['Journey'] == (journey+1)]
    matches_J
    
    vs = []
    for team in df_1.Team:

        for index in matches_J.index:
           
            if team == matches_J.loc[index].Home:
            
                vs.append(matches_J.loc[index].Away)
                
            if team == matches_J.loc[index].Away:
                
                vs.append(matches_J.loc[index].Home)
    
    df_1['vs'] = vs
    
    points_per_vs_team = comunio.groupby('Team').sum().reset_index()
    points_per_vs_team= points_per_vs_team.rename(columns={'Total_Points': 'Vs_Squad_Points',
                                                           'Team':'Vs_Team',
                                                           'Points_Average': 'Vs_Squad_Average_Points',
                                                           'Avg_last_5_games': 'Vs_Squad_Avg_last_5_Games',
                                                           'Value':'Vs_Value_Squad',
                                                           f'J_{journey -4}': f'Vs_Squad_Points_J_{journey -4}',
                                                           f'J_{journey -3}': f'Vs_Squad_Points_J_{journey -3}',
                                                           f'J_{journey -2}': f'Vs_Squad_Points_J_{journey -2}',
                                                           f'J_{journey -1}': f'Vs_Squad_Points_J_{journey -1}',
                                                           f'J_{journey}': f'Vs_Squad_Points_J_{journey}',})
    
    points_per_vs_team = points_per_vs_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_2 = df_1.merge(points_per_vs_team, how='left', left_on='vs', right_on='Vs_Team')
    
    team_clas = []

    for team in comunio.Team:
        for index in clas.index:
            
            if team in clas.loc[index].Team:
                
                team_clas.append(clas.loc[index].Position)
    
    team_clas_vs = []

    for team in df_2.Vs_Team:
        for index in clas.index:
           
            if team in clas.loc[index].Team:
                
                team_clas_vs.append(clas.loc[index].Position)
    
    df_2['Team_clas'] = team_clas
    df_2['Vs_Team_clas'] = team_clas_vs
    
    
    return df_2.drop('Team_id', axis=1)

In [6]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test 

In [7]:
j_30 = create_df(30)
j_31 = create_df(31)

In [9]:
X, y, X_train, X_test,y_train,y_test = preprocess_data(j_30,j_31,31)

In [12]:
rfr = RandomForestRegressor()
xgbr = XGBRegressor()
gb = GradientBoostingRegressor()
models = (rfr,xgbr,gb)

In [13]:
for model in models:
    name = str(model)[:13]
    if 'Sequential' in name:
        
        model.fit(X_train_s,
                   y_train_s,
                   epochs=50,
                   validation_data=0.15)
        pred = model.predict(X_test_s)
        pred2 = y_scaler.inverse_transform(pred)
        mse = mean_squared_error(pred2,y_test, squared=False)
        r2 = r2_score(pred2, y_test)
        
        pd.DataFrame(model.history).plot(figsize=(8,5))
        plt.grid()
        plt.show()
        
        
    if 'MLP' in name:
        
        model.fit(X_train_s,y_train_s)
        model.score(X_train_s, y_train_s)
        pred = model.predict(X_test_s)
        pred_test = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred_test,y_test, squared=False)
        
        
    else:
        model.fit(X_train,y_train)
        model.score(X_train, y_train)
        pred = model.predict(X_test)
        #pred2 = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred,y_test, squared=False)
        r2 = r2_score(pred, y_test)
    
    print (f'mse del modelo {name} = {mse}')

mse del modelo RandomForestR = 2.914981537010331
mse del modelo XGBRegressor( = 2.9855435917849116
mse del modelo GradientBoost = 2.8899211724520346


In [14]:
pred_rfr = rfr.predict(X)
pred_xgb = xgbr.predict(X)
pred_gb = gb.predict(X)

In [18]:
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds = round(preds)
preds

,Player,Position,rfr,xgb,gb
0,Unai Simón,PT,3.0,3.0,4.0
1,Agirrezabala,PT,0.0,-0.0,0.0
2,Ezkieta,PT,0.0,-0.0,0.0
3,Iñigo Martínez,DF,1.0,0.0,2.0
4,Daniel Vivian,DF,3.0,3.0,3.0
5,Lekue,DF,1.0,0.0,1.0
6,De Marcos,DF,2.0,3.0,3.0
7,Yeray Álvarez,DF,3.0,3.0,3.0
8,Balenziaga,DF,4.0,6.0,4.0
9,Yuri Berchiche,DF,4.0,4.0,4.0


In [22]:
squad = ['Courtois', 'Bono',
         'Balliu', 'Fran García','Ronald Araujo', 'Alderete', 'Manu Sánchez', 'Giménez',
         'Yéremi',  'Gavi',  'Modric',  'Brais', 'Papu Gómez', 'Dani Rodríguez', 'Pere Milla', 'Cervi', 'Berenguer',
         'Vinícius Júnior', 'Aubameyang', 'Sancet', 'João Félix', 'Morales']
df_squad_kike = pd.DataFrame()
for player in squad:
    #print (player)
    for p in preds.Player:
        if player in p:
            #print('encontrado')
            add_player = pd.DataFrame(preds.loc[preds.Player==p])
            #print(add_player)
            df_squad_kike = df_squad_kike.append(add_player)
print(len(squad),len(df_squad_kike))

df_squad_kike.at[514,'Position'] = 'MD'
round(df_squad_kike)

22 22


,Player,Position,rfr,xgb,gb
199,Courtois,PT,6.0,7.0,5.0
545,Bono,PT,6.0,3.0,5.0
233,Balliu,DF,6.0,5.0,6.0
234,Fran García,DF,3.0,3.0,4.0
109,Ronald Araujo,DF,5.0,7.0,5.0
410,Omar Alderete,DF,3.0,4.0,3.0
53,Manu Sánchez,DF,5.0,6.0,4.0
27,José Giménez,DF,1.0,0.0,2.0
393,Yéremi Pino,MD,2.0,2.0,3.0
120,Gavi,MD,5.0,5.0,4.0


In [25]:
def once_ideal_xbgr(data):
    gk = data.loc[data['Position']=='PT'].sort_values('xgb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('xgb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('xgb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('xgb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [26]:
xgb_eleven = round(once_ideal_xbgr(df_squad_kike))
xgb_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,6.0,7.0,5.0
109,Ronald Araujo,DF,5.0,7.0,5.0
53,Manu Sánchez,DF,5.0,6.0,4.0
233,Balliu,DF,6.0,5.0,6.0
120,Gavi,MD,5.0,5.0,4.0
528,Brais Méndez,MD,4.0,4.0,3.0
530,Franco Cervi,MD,4.0,4.0,5.0
514,Pere Milla,MD,2.0,3.0,3.0
221,Vinícius Júnior,DL,8.0,11.0,10.0
129,Aubameyang,DL,8.0,9.0,9.0


In [28]:
def once_ideal_rf(data):
    gk = data.loc[data['Position']=='PT'].sort_values('rfr', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('rfr', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('rfr', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('rfr', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [29]:
def once_ideal_gb(data):
    gk = data.loc[data['Position']=='PT'].sort_values('gb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('gb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('gb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('gb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [30]:
rf_eleven = round(once_ideal_rf(df_squad_kike))
rf_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,6.0,7.0,5.0
233,Balliu,DF,6.0,5.0,6.0
109,Ronald Araujo,DF,5.0,7.0,5.0
53,Manu Sánchez,DF,5.0,6.0,4.0
120,Gavi,MD,5.0,5.0,4.0
528,Brais Méndez,MD,4.0,4.0,3.0
530,Franco Cervi,MD,4.0,4.0,5.0
14,Álex Berenguer,MD,3.0,2.0,3.0
221,Vinícius Júnior,DL,8.0,11.0,10.0
129,Aubameyang,DL,8.0,9.0,9.0


In [31]:
gb_eleven = round(once_ideal_gb(df_squad_kike))
gb_eleven

,Player,Position,rfr,xgb,gb
199,Courtois,PT,6.0,7.0,5.0
233,Balliu,DF,6.0,5.0,6.0
109,Ronald Araujo,DF,5.0,7.0,5.0
234,Fran García,DF,3.0,3.0,4.0
530,Franco Cervi,MD,4.0,4.0,5.0
120,Gavi,MD,5.0,5.0,4.0
393,Yéremi Pino,MD,2.0,2.0,3.0
528,Brais Méndez,MD,4.0,4.0,3.0
221,Vinícius Júnior,DL,8.0,11.0,10.0
129,Aubameyang,DL,8.0,9.0,9.0
